## A notebook to seamlessly take `BLAST` output to GO slim list

Set working directory

In [1]:
wd = "../analyses/BLAST_to_GOslim/"

In [2]:
cd $wd

/Users/graciecrandall/Documents/GitHub/project-crab/analyses/BLAST_to_GOslim


### In this directory you will need three files:
1. blastout file in format -6
2. Uniprot GO annotation file (340M) available here: http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted
3. GOslim file available here: http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

In [3]:
!curl https://gannet.fish.washington.edu/Atumefaciens/20200408_cbai_diamond_blastx_megan/20200408.C_bairdi.megan.Trinity.blastx.outfmt6 \
-o 20200408.C_bairdi.megan.Trinity.blastx.outfmt6

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1997k  100 1997k    0     0  3119k      0 --:--:-- --:--:-- --:--:-- 3426k


In [4]:
!curl http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted -o uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  21.8M      0  0:00:15  0:00:15 --:--:-- 24.1M


In [5]:
!curl http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted -o GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  1092k      0  0:00:02  0:00:02 --:--:-- 2874k


In [6]:
#check that files are in directory
!ls

20200408.C_bairdi.megan.Trinity.blastx.outfmt6
GO-GOslim.sorted
readme.md
uniprot-SP-GO.sorted


### Set the blastout file as a variable

In [7]:
blastout="20200408.C_bairdi.megan.Trinity.blastx.outfmt6"

## That should be the last thing you have to type!

In [8]:
!head -2 $blastout

TRINITY_DN21551_c1_g1_i1	sp|Q5ZK63|TCP4_CHICK	45.1	71	38	1	410	201	56	126	1.2e-12	75.1
TRINITY_DN21560_c0_g1_i1	sp|Q56JU9|RS24_BOVIN	74.2	128	33	0	68	451	4	131	2.4e-44	179.9


In [9]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [10]:
!head -2 _blast-sep.tab

TRINITY_DN21551_c1_g1_i1	sp	Q5ZK63	TCP4_CHICK	45.1	71	38	1	410	201	56	126	1.2e-12	75.1
TRINITY_DN21560_c0_g1_i1	sp	Q56JU9	RS24_BOVIN	74.2	128	33	0	68	451	4	131	2.4e-44	179.9


In [11]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [12]:
!head -2 _blast-sort.tab

A0A068FIK2	TRINITY_DN6827_c0_g1_i1	4.5e-30
A0A087WPF7	TRINITY_DN15151_c0_g1_i1	3.2e-06


In [13]:
!head -2 uniprot-SP-GO.sorted

A0A023GPI8	LECA_CANBL	reviewed	Lectin alpha chain (CboL) [Cleaved into: Lectin beta chain; Lectin gamma chain]		Canavalia boliviana	237			mannose binding [GO:0005537]; metal ion binding [GO:0046872]	mannose binding [GO:0005537]; metal ion binding [GO:0046872]	GO:0005537; GO:0046872
A0A023GPJ0	CDII_ENTCC	reviewed	Immunity protein CdiI	cdiI ECL_04450.1	Enterobacter cloacae subsp. cloacae (strain ATCC 13047 / DSM 30054 / NBRC 13535 / NCDC 279-56)	145					


In [14]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [15]:
!head -2 _blast-annot.tab

A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0003777; GO:0005524; GO:0005737; GO:0007018; GO:0008017; GO:0055028; GO:0071555
A0A0F7YZI5	TRINITY_DN7195_c0_g1_i1	GO:0005179; GO:0005576


### The following is a script modified from Sam White

In [16]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [17]:
!head _blast-GO-unfolded.tab

A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0003777
A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0005524
A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0005737
A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0007018
A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0008017
A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0055028
A0A068FIK2	TRINITY_DN6827_c0_g1_i1	GO:0071555
A0A0F7YZI5	TRINITY_DN7195_c0_g1_i1	GO:0005179
A0A0F7YZI5	TRINITY_DN7195_c0_g1_i1	GO:0005576
A0A0F7Z3J2	TRINITY_DN25564_c0_g1_i1	GO:0005179


In [18]:
#gets rid of lines with no GOIDs
!sort -k3 _blast-GO-unfolded.tab | grep "GO:" > _blast-GO-unfolded.sorted

In [19]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 3 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $3, $1, $5, $6}' \
> Blastquery-GOslim.tab

In [20]:
!head Blastquery-GOslim.tab

TRINITY_DN4563_c0_g1_i1	GO:0000001	cell organization and biogenesis	P
TRINITY_DN2020_c0_g1_i5	GO:0000002	cell organization and biogenesis	P
TRINITY_DN4388_c0_g1_i3	GO:0000002	cell organization and biogenesis	P
TRINITY_DN4388_c0_g1_i4	GO:0000002	cell organization and biogenesis	P
TRINITY_DN4985_c0_g1_i1	GO:0000002	cell organization and biogenesis	P
TRINITY_DN22531_c0_g1_i1	GO:0000002	cell organization and biogenesis	P
TRINITY_DN4468_c0_g2_i1	GO:0000002	cell organization and biogenesis	P
TRINITY_DN9608_c0_g1_i1	GO:0000002	cell organization and biogenesis	P
TRINITY_DN14285_c0_g1_i1	GO:0000002	cell organization and biogenesis	P
TRINITY_DN1568_c1_g1_i14	GO:0000002	cell organization and biogenesis	P


In [21]:
!cat Blastquery-GOslim.tab | grep "	P"  | awk -F $'\t' '{print $4}' | sort | uniq -c

114708 P


In [22]:
!cat Blastquery-GOslim.tab | grep "	P" | awk -F $'\t' '{print $3}' | sort | uniq -c | sort -r

8350 transport
8282 protein metabolism
6622 signal transduction
5317 stress response
4719 cell cycle and proliferation
2887 DNA metabolism
2093 death
20063 developmental processes
1701 cell-cell signaling
16874 other biological processes
14568 cell organization and biogenesis
1426 cell adhesion
11074 other metabolic processes
10732 RNA metabolism


In [23]:
!cat Blastquery-GOslim.tab | grep "	P" | awk -F $'\t' '{print $3}' | sort | uniq -c | sort -r > GOslim-P-pie.txt